<a href="https://colab.research.google.com/github/michaelgetachew-abebe/AirBnB_clone/blob/main/00_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -qU langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 8.6 MB/s eta 0:00:00


In [4]:
!pip install -qU huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 2.8 MB/s eta 0:00:00


In [5]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.environ["API_KEY"]

In [6]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
falcon = HuggingFaceHub(
    repo_id="tiiuae/falcon-7b-instruct",
    model_kwargs={"temperature":0.2}
)

In [7]:
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

In [8]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=falcon
)

In [9]:
question = "Where is Ethiopia located?"

In [10]:
print(llm_chain(question))

{'question': 'Where is Ethiopia located?', 'text': 'Question: Where is Ethiopia located?\n\nAnswer: \nEthiopia is located in East Africa, bordering Djibouti, Somalia, and Eritrea.'}


In [11]:
# Asking multiple Questions
qs = [
    {'question': "Where is Ethiopia located?"},
    {'question': "Where is the arc of the covenant?"},
    {'question': "How old is the moon?"},
    {'question': "What is amharic?"}
]

In [12]:
res = llm_chain.generate(qs)
res

LLMResult(generations=[[Generation(text='Question: Where is Ethiopia located?\n\nAnswer: \nEthiopia is located in East Africa, bordering Djibouti, Somalia, and Eritrea.')], [Generation(text='Question: Where is the arc of the covenant?\n\nAnswer: \nThe Ark of the Covenant is a sacred chest containing the tablets of the Ten Commandments, which were written by God on two tablets of stone. It is located in the Ark of the Covenant, which is housed in the Temple of the Ark of the Covenant in Jerusalem, Israel.')], [Generation(text='Question: How old is the moon?\n\nAnswer: 4.5 billion years old.')], [Generation(text='Question: What is amharic?\n\nAnswer: \nAmharic is a language of the Amhara people of Ethiopia. It is also the official language of the country. It is a member of the East Cushitic language family and is closely related to Tigrinya and Oromo. It is spoken by over 30 million people in Ethiopia and is also spoken in Eritrea.')]], llm_output=None, run=[RunInfo(run_id=UUID('61381202

In [13]:
# Also all the questions can be feed all one time but get answered one by one
multiple_question_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""

multiple_question_prompt = PromptTemplate(
    template=multiple_question_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=multiple_question_prompt,
    llm=falcon
)

In [14]:
questions = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
    )

In [16]:
print(llm_chain(questions))

{'questions': 'Which NFL team won the Super Bowl in the 2010 season?\nIf I am 6 ft 4 inches, how tall am I in centimeters?\nWho was the 12th person on the moon?How many eyes does a blade of grass have?', 'text': 'Answer the following questions one at a time.\n\nQuestions:\nWhich NFL team won the Super Bowl in the 2010 season?\nIf I am 6 ft 4 inches, how tall am I in centimeters?\nWho was the 12th person on the moon?How many eyes does a blade of grass have?\n\nAnswers:\n1. New England Patriots\n2. 6 ft 4 in\n3. Neil Armstrong\n4. 12'}


In [28]:
sql_coder = HuggingFaceHub(
    repo_id="defog/sqlcoder",
    model_kwargs={"temperature": 0.1}
)

In [29]:
# ### Task
# Generate a SQL query to answer [QUESTION]{user_question}[/QUESTION]

# ### Database Schema
# The query will run on a database with the following schema:
# {table_metadata_string_DDL_statements}

# ### Answer
# Given the database schema, here is the SQL query that [QUESTION]{user_question}[/QUESTION]
# [SQL]
sql_prompt_template = """
Generate a SQL query to answer [QUESTION]{user_question}[/QUESTION]
The query will run on a database with the following schema:

CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

Given the database schema, here is the SQL query that [QUESTION]{user_question}[/QUESTION]
[SQL]
"""

In [30]:
sql_prompt = PromptTemplate(template=sql_prompt_template, input_variables=["user_question"])

In [31]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=sql_coder
)

In [32]:
question = "How may products are there?"

In [33]:
print(llm_chain(question))

HfHubHTTPError:  (Request ID: taypFXQzleUH-CmosT6yq)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/defog/sqlcoder.
If you are trying to create or update content,make sure you have a token with the `write` role.
The model defog/sqlcoder is too large to be loaded automatically (31GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).